In [ ]:
import numpy as np
from math import factorial
import matplotlib.pyplot as plt
from matplotlib import animation

In [ ]:
def hermite(x, n):
    """
    hermite polynomials of order n
    defined on the real space x

    Parameters
    ----------
    x : np.ndarray
        Real space
    n : int
        order of the polynomial

    Returns
    -------
    np.ndarray
        Hermite polynomial of order n
    """
    herm_coeffs = np.zeros(n+1)
    herm_coeffs[n] = 1
    return np.polynomial.hermite.hermval(x, herm_coeffs)

def stationary_state(x,n):
    """
    Returns the stationary state of order
    n of the quantum harmonic oscillator

    Parameters
    ----------
    x : np.ndarray
        Real space
    n : int
        order of the stationary state. 0 is ground state.

    Returns
    -------
    np.ndarray
        Stationary state of order n
    """
    prefactor = 1./np.sqrt(2.**n * factorial(n)) * (1/(np.pi))**(0.25)
    psi = prefactor * np.exp(- x**2 / 2) * hermite(x,n)
    return psi

In [ ]:
class Param:
    """
    Container for holding all simulation parameters

    Parameters
    ----------
    xmax : float
        The real space is between [-xmax, xmax]
    num_x : int
        Number of intervals in [-xmax, xmax]
    dt : float
        Time discretization
    timesteps : int
        Number of timestep
    im_time : bool, optional
        If True, use imaginary time evolution.
        Default to False.
    """
    def __init__(self,
                 xmax: float,
                 num_x: int,
                 dt: float,
                 timesteps: int,
                 im_time: bool = False) -> None:

        self.xmax = xmax
        self.num_x = num_x
        self.dt = dt
        self.timesteps = timesteps
        self.im_time = im_time

        self.dx = 2 * xmax / num_x
        # Real time grid
        self.x = np.arange(-xmax + xmax / num_x, xmax, self.dx)
        # Definition of the momentum
        self.dk = np.pi / xmax
        # Momentum grid -> DONE FOR THE FFT, which wants the frequencies in this order!!!
        self.k = np.concatenate((np.arange(0, num_x / 2), np.arange(-num_x / 2, 0))) * self.dk

class Operators:
    """Container for holding operators and wavefunction coefficients."""
    def __init__(self, res: int) -> None:

        self.V = np.empty(res, dtype=complex)
        self.R = np.empty(res, dtype=complex)
        self.K = np.empty(res, dtype=complex)
        self.wfc = np.empty(res, dtype=complex)

In [ ]:
def init(par: Param, voffset: float, wfcoffset: float, n:int = 0) -> Operators:
    """
    Initialize the wavefunction coefficients and the potential.

    Parameters
    ----------
    par: Param
        Class containing the parameters of the simulation
    voffset : float, optional
        Offset of the quadratic potential in real space
        Default to 0.
    wfcoffset: float, optional
        Offset of the wavefunction in real space. Default to 0.
    n : int, optional
        Order of the hermite polynomial (i.e. level of eigenstate)

    Returns
    -------
    Operators
        Filled operators
    """
    opr = Operators(len(par.x))

    # Quadratic potential
    opr.V = 0.5 * (par.x - voffset) ** 2
    opr.wfc = stationary_state(par.x-wfcoffset, n).astype(complex)

    # Complete here to implement imaginary time evolution based on par.im_time
    coeff = 1 if par.im_time else 1j

    # Operator in momentum space -> potential
    opr.K = np.exp(-0.5 * (par.k ** 2) * par.dt * coeff)
    # Operator in real space -> kinetic
    opr.R = np.exp(-0.5 * opr.V * par.dt * coeff)

    return opr


In [ ]:
def split_op(par: Param, opr: Operators) -> None:
    """
    Split operator method for time evolution.
    Works in place

    Parameters
    ----------
    par : Param
        Parameters of the simulation
    opr : Operators
        Operators of the simulation

    Returns
    -------
    None
        Acts in place
    """
    # Store the results in res.
    # Use the first 100 to store the wfc in real
    # space and the other half to store the wavefunction
    # in momentum space
    res = np.zeros((100, 2*par.num_x))
    jj = 0
    for i in range(par.timesteps):

        # Half-step in real space
        opr.wfc *= opr.R

        # Step in momentum space
        opr.wfc = np.fft.fft(opr.wfc)
        opr.wfc *= opr.K
        opr.wfc = np.fft.ifft(opr.wfc)

        # Final half-step in real space
        opr.wfc *= opr.R

        # Density for plotting and potential
        density = np.abs(opr.wfc) ** 2

        # Renormalizing for imaginary time
        if par.im_time:
            renorm_factor = np.sum(density * par.x)
            opr.wfc /= np.sqrt(renorm_factor)

        # This is set to save exactly 100 snapshots, no
        # matter how many timesteps were specified.
        if i % (par.timesteps // 100) == 0:
            res[jj, 0:par.num_x] = np.real(density)
            res[jj, par.num_x:2*par.num_x] = np.abs(np.fft.fft(opr.wfc))**2
            jj += 1

    return res

In [ ]:
def calculate_energy(par: Param, opr: Operators) -> float:
    """Calculate the energy <Psi|H|Psi>."""
    # Creating real, momentum, and conjugate wavefunctions.
    wfc_r = opr.wfc
    wfc_k = np.fft.fft(wfc_r)
    wfc_c = np.conj(wfc_r)

    # Finding the momentum and real-space energy terms
    energy_k = 0.5 * wfc_c * np.fft.ifft((par.k ** 2) * wfc_k)
    energy_r = wfc_c * opr.V * wfc_r

    # Integrating over all space
    energy_final = sum(energy_k + energy_r).real

    return energy_final * par.dx